In [ ]:
### Importacion de librerias
import hashlib
import time

In [ ]:
### Definicion de funcion para hashear una cadenas de texto

def hash_tx(tx):
  hash_object = hashlib.sha256(tx.encode())
  hash_hex = hash_object.hexdigest()
  return hash_hex

In [ ]:
### se crea la funcion de arbol de merkel

def create_markle_root(transactions):
  hashed_transactions = [hash_tx(tx) for tx in transactions]

  while len(hashed_transactions) > 1:
    if len(hashed_transactions) % 2 != 0:
      hashed_transactions.append(hashed_transactions[-1])
    new_lvl = []
    for i in range(0, len(hashed_transactions), 2):
      concatenate_hasheds = hashed_transactions[i] + hashed_transactions[i + 1]
      new_hash = hash_tx(concatenate_hasheds)
      new_lvl.append(new_hash)

    hashed_transactions = new_lvl
  return hashed_transactions

In [ ]:
class Block:
  def __init__(self, previous_hash, transactions):
    self.previous_hash = previous_hash
    self.transactions = transactions
    self.timestamp = time.time()
    self.merkle_root = create_markle_root(transactions)
    self.nonce = 0
    self.hash = self.calculate_hash()

  def calculate_hash(self):
    block_content = str(self.previous_hash) + str(self.merkle_root) + str(self.timestamp) + str(self.nonce)
    return hash_tx(block_content)

  def mine_block(self, difficulty):
    required_prefix = "0" * difficulty
    while not self.hash.startswith(required_prefix):
      self.nonce += 1
      self.hash = self.calculate_hash()
    print("Bloque minado:" + self.hash)

In [ ]:
### Lista de transacciones

transactions1 = [
    "Transacción 1",
    "Transacción 2",
    "Transacción 3",
    "Transacción 4"
]

In [ ]:
### creacion del bloque genesis
genesis_block = Block("0", transactions1)

In [ ]:
###Imprime las vairables dentro del block genesis
print(vars(genesis_block))

{'previous_hash': '0', 'transactions': ['Transacción 1', 'Transacción 2', 'Transacción 3', 'Transacción 4'], 'timestamp': 1725233385.5907888, 'merkle_root': ['7085d44355b195a817fe637158bae4c7783f4353ec61774bc6d80e0b17636b1c'], 'nonce': 0, 'hash': '724eb2b2dc4b4e114fa7aa6f38d8630d1fb8c7b8f7c0873176351e0ad8448db7'}


In [ ]:
###Calcula el hash bloque genesis
genesis_block.mine_block(5)
print(genesis_block.hash)

Bloque minado: {self.hash}
000004d0739c41884baf699180f354b43e7a4859ee76d07192e692314e6b00de


In [ ]:
class Blockchain:
  def __init__(self, difficulty):
    self.chain = []  # Lista que contendrá todos los bloques
    self.difficulty = difficulty # Puedes ajustar la dificultad según lo necesites

    # Crear el bloque génesis (el primer bloque de la cadena)
    genesis_block = Block("0", ["Genesis Block"])
    genesis_block.mine_block(self.difficulty)
    self.chain.append(genesis_block)

  def get_latest_block(self):
    # Retorna el último bloque de la cadena
    return self.chain[-1]

  def add_block(self, new_block):
    # Establecer el hash del bloque anterior
    new_block.previous_hash = self.get_latest_block().hash

    # Minar el nuevo bloque
    new_block.mine_block(self.difficulty)

    # Agregar el nuevo bloque a la cadena
    self.chain.append(new_block)

  def is_chain_valid(self):
    # Verifica la integridad de la cadena de bloques
    for i in range(1, len(self.chain)):
        current_block = self.chain[i]
        previous_block = self.chain[i - 1]

        # Verificar que el hash actual es correcto
        if current_block.hash != current_block.calculate_hash():
            print("Hash incorrecto en el bloque:", i)
            return False

        # Verificar que el bloque apunta correctamente al bloque anterior
        if current_block.previous_hash != previous_block.hash:
            print("Enlace roto entre bloques:", i)
            return False

    return True


In [ ]:
# Crear una instancia de la blockchain
my_blockchain = Blockchain(5)

# Agregar nuevos bloques a la blockchain
new_block_1 = Block(my_blockchain.get_latest_block().hash, ["Transacción 1", "Transacción 2"])
my_blockchain.add_block(new_block_1)

new_block_2 = Block(my_blockchain.get_latest_block().hash, ["Transacción 3", "Transacción 4"])
my_blockchain.add_block(new_block_2)

Bloque minado:00000a12434a241e6025254b8c1410bacb20c151990fec36e2d0206d00a5d276
Bloque minado:000008c9a0d0b69e3e193bfedb3b8eef46c09d1ac2e5ce88e963989181a97efe
Bloque minado:000009e4f13a05e9dd170d496d30f1e22fd5321dd9a0e43f7ec7ae7f3dcca4b4
